In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch import tensor

import torchaudio
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np

import librosa

In [5]:
#%cd /content/drive/MyDrive/Colab Notebooks/sound classification

/content/drive/MyDrive/Colab Notebooks/sound classification


In [2]:
# 定义一个自定义数据集类
class AudioDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        #self.label_list = label_list

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = self.file_list[idx]
        #label = self.label_list[idx]

        # 使用 Librosa 加载音频文件，并将其转换为梅尔频谱图
        y, sr = librosa.load(file_path, sr=16000)
        mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=1024, hop_length=512, n_mels=80)
        mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
        mel_spec_db = np.expand_dims(mel_spec_db, axis=0)

        # 返回梅尔频谱图
        return mel_spec_db


In [3]:
idx_to_label = "Red-billed Starling,Intermediate Egret,Blue-and-white Flycatcher,Pin-tailed Snipe,Eastern Marsh-Harrier,Manchurian Reed Warbler,Chinese Pond-Heron,Rock Bunting,Isabelline Shrike,Japanese Scops-Owl,Red-backed Shrike,Bronzed Drongo,Claudia's Leaf Warbler,Common Myna,Koklass Pheasant,Barred Warbler,Besra,Pallid Harrier,Tickell's Leaf Warbler,Gray-cheeked Warbler".split(',')

NUM_CLASSES = len(idx_to_label)

label_to_idx = {idx_to_label[i]: i for i in range(NUM_CLASSES)}

test_data_path = 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy'

label_to_idx = {value: key for key, value in label_to_idx.items()}

In [4]:
label_to_idx

{0: 'Red-billed Starling',
 1: 'Intermediate Egret',
 2: 'Blue-and-white Flycatcher',
 3: 'Pin-tailed Snipe',
 4: 'Eastern Marsh-Harrier',
 5: 'Manchurian Reed Warbler',
 6: 'Chinese Pond-Heron',
 7: 'Rock Bunting',
 8: 'Isabelline Shrike',
 9: 'Japanese Scops-Owl',
 10: 'Red-backed Shrike',
 11: 'Bronzed Drongo',
 12: "Claudia's Leaf Warbler",
 13: 'Common Myna',
 14: 'Koklass Pheasant',
 15: 'Barred Warbler',
 16: 'Besra',
 17: 'Pallid Harrier',
 18: "Tickell's Leaf Warbler",
 19: 'Gray-cheeked Warbler'}

In [5]:
test_files=[]
#train_labels =[]
for wav_file in tqdm(os.listdir(test_data_path)):
    test_files.append(test_data_path + f'/{wav_file}')

100%|████████████████████████████████████████████████████████████████████████████| 189/189 [00:00<00:00, 188653.84it/s]


In [6]:
test_files

['C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST001.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST002.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST003.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST004.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST005.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST006.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST007.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST008.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST009.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST010.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST011.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST012.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST013.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST014.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST015.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST016.ogg',
 'C:/Users/maple/研究生杂学/声音检测/data/test1_easy/TEST017.ogg',
 'C:/Users/map

In [7]:
test_dataset = AudioDataset(test_files)
test_loader = DataLoader(test_dataset, shuffle=False)

In [8]:
len(test_loader)

189

In [21]:
class AudioCNN(nn.Module):
    def __init__(self):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2), stride=(2,2))

        self.fc1 = nn.Linear(39680, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.relu3 = nn.ReLU(inplace=True)

        self.fc2 = nn.Linear(128, 20)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.fc2(x)

        return x


In [22]:
model = torch.load('models/CNN.pt')

In [23]:
model

AudioCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=49920, out_features=128, bias=True)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (fc2): Linear(in_features=128, out_features=20, bias=True)
)

In [11]:
import pandas as pd
result = pd.read_csv(r'C:\Users\maple\研究生杂学\声音检测\data\test1_FileList.csv')

In [12]:
result

,FileName
0,TEST001.ogg
1,TEST002.ogg
2,TEST003.ogg
3,TEST004.ogg
4,TEST005.ogg
...,...
184,TEST185.ogg
185,TEST186.ogg
186,TEST187.ogg
187,TEST188.ogg


In [13]:
result['Predicted_ID'] = None

In [14]:
result

,FileName,Predicted_ID
0,TEST001.ogg,None
1,TEST002.ogg,None
2,TEST003.ogg,None
3,TEST004.ogg,None
4,TEST005.ogg,None
...,...,...
184,TEST185.ogg,None
185,TEST186.ogg,None
186,TEST187.ogg,None
187,TEST188.ogg,None


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [35]:
for idx, inputs in enumerate(test_loader):
    print(inputs.shape)
    a=inputs.shape[-1]
    print(a)
    break

torch.Size([1, 1, 80, 157])
157


In [24]:
model.eval()

AudioCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU(inplace=True)
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU(inplace=True)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=49920, out_features=128, bias=True)
  (bn3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU(inplace=True)
  (fc2): Linear(in_features=128, out_features=20, bias=True)
)

In [36]:
for idx, inputs in enumerate(test_loader):
    inputs = inputs.to(device)
    print(inputs.shape)
    if(inputs.shape[-1]==157):
        outputs = model(inputs)
        pred = outputs.argmax(dim=1, keepdim=True)
        print(pred)
        pred_int = pred.item()
        result.iloc[idx,1] = pred_int
result

torch.Size([1, 1, 80, 157])
tensor([[13]], device='cuda:0')
torch.Size([1, 1, 80, 125])
torch.Size([1, 1, 80, 157])
tensor([[8]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[3]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[16]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[1]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[11]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[16]], device='cuda:0')
torch.Size([1, 1, 80, 110])
torch.Size([1, 1, 80, 157])
tensor([[11]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[16]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[12]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[13]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[13]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[12]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[18]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[8]], device='cuda:0')
torch.Size([1, 1, 80, 157])
tensor([[8]], device

,FileName,Predicted_ID
0,TEST001.ogg,13
1,TEST002.ogg,None
2,TEST003.ogg,8
3,TEST004.ogg,3
4,TEST005.ogg,16
...,...,...
184,TEST185.ogg,1
185,TEST186.ogg,3
186,TEST187.ogg,8
187,TEST188.ogg,11


In [37]:
for i in range(189):
    if(result.iloc[i,1]==0):
        result.iloc[i,1]=20
result

,FileName,Predicted_ID
0,TEST001.ogg,13
1,TEST002.ogg,None
2,TEST003.ogg,8
3,TEST004.ogg,3
4,TEST005.ogg,16
...,...,...
184,TEST185.ogg,1
185,TEST186.ogg,3
186,TEST187.ogg,8
187,TEST188.ogg,11


In [39]:
result.to_csv('test_easy.csv', index=False)

RestNet18

In [40]:
#残差模型
import torch
import torch.nn as nn
import torch.nn.functional as F
 
#定义残差块ResBlock
class ResBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResBlock, self).__init__()
        #残差块内连续的2个卷积层
        self.left = nn.Sequential(
            nn.Conv2d(inchannel, outchannel, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(inplace=True),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.shortcut = nn.Sequential()
        if stride != 1 or inchannel != outchannel:
            #shortcut，这里为了跟2个卷积层的结果结构一致，要做处理
            self.shortcut = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel)
            )
            
    def forward(self, x):
        out = self.left(x)
        #将2个卷积层的输出跟处理过的x相加，实现ResNet的基本结构
        out = out + self.shortcut(x)
        out = F.relu(out)
        
        return out
 
#实现ResNet-18模型
class ResNet(nn.Module):
    def __init__(self, ResBlock, num_classes=20):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU()
        )
        self.layer1 = self.make_layer(ResBlock, 64, 2, stride=1)
        self.layer2 = self.make_layer(ResBlock, 128, 2, stride=2)
        self.layer3 = self.make_layer(ResBlock, 256, 2, stride=2)        
        self.layer4 = self.make_layer(ResBlock, 512, 2, stride=2)        
        self.fc = nn.Linear(5120, num_classes)
    #这个函数主要是用来，重复同一个残差块    
    def make_layer(self, block, channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.inchannel, channels, stride))
            self.inchannel = channels
        return nn.Sequential(*layers)
    
    def forward(self, x):
        #在这里，整个ResNet18的结构就很清晰了
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [44]:
model = torch.load('ResNet18.pt')
model.eval()
for idx, inputs in enumerate(test_loader):
    inputs = inputs.to(device)
    #print(inputs.shape)
    if(inputs.shape[-1]==157):
        outputs = model(inputs)
        pred = outputs.argmax(dim=1, keepdim=True)
        print(pred)
        pred_int = pred.item()
        result.iloc[idx,1] = pred_int
for i in range(189):
    if(result.iloc[i,1]==0):
        result.iloc[i,1]=20
result.to_csv('test_easy.csv', index=False)

tensor([[13]], device='cuda:0')
tensor([[15]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[19]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[15]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[18]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[19]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[9]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[11]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[18]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[14]], device='cuda:0')
tensor([[17]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[2]], device='

In [45]:
model = torch.load('models/ResNet50-50.pt')

In [46]:
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [47]:
for idx, inputs in enumerate(test_loader):
    inputs = inputs.to(device)
    #print(inputs.shape)
    if(inputs.shape[-1]==157):
        outputs = model(inputs)
        pred = outputs.argmax(dim=1, keepdim=True)
        print(pred)
        pred_int = pred.item()
        result.iloc[idx,1] = pred_int
result

tensor([[13]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[8]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[19]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[18]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[9]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[15]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[18]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[0]], device='cuda:0')
tensor([[9]], device

,FileName,Predicted_ID
0,TEST001.ogg,13
1,TEST002.ogg,None
2,TEST003.ogg,12
3,TEST004.ogg,3
4,TEST005.ogg,16
...,...,...
184,TEST185.ogg,1
185,TEST186.ogg,17
186,TEST187.ogg,1
187,TEST188.ogg,11


In [48]:
result.to_csv('test_easy3.csv', index=False)

In [49]:
for idx, inputs in enumerate(test_loader):
    inputs = inputs.to(device)
    #print(inputs.shape)
    outputs = model(inputs)
    pred = outputs.argmax(dim=1, keepdim=True)
    print(pred)
    pred_int = pred.item()
    result.iloc[idx,1] = pred_int
result

tensor([[13]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[8]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[19]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[18]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[9]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[15]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[18]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[12]], devic

,FileName,Predicted_ID
0,TEST001.ogg,13
1,TEST002.ogg,1
2,TEST003.ogg,12
3,TEST004.ogg,3
4,TEST005.ogg,16
...,...,...
184,TEST185.ogg,1
185,TEST186.ogg,17
186,TEST187.ogg,1
187,TEST188.ogg,11


In [50]:
result.to_csv('test_easy4.csv', index=False)

In [51]:
for i in range(189):
    if(result.iloc[i,1]==0):
        result.iloc[i,1]=20
result

,FileName,Predicted_ID
0,TEST001.ogg,13
1,TEST002.ogg,1
2,TEST003.ogg,12
3,TEST004.ogg,3
4,TEST005.ogg,16
...,...,...
184,TEST185.ogg,1
185,TEST186.ogg,17
186,TEST187.ogg,1
187,TEST188.ogg,11


In [52]:
result.to_csv('test_easy5.csv', index=False)

In [9]:
model = torch.load('models/ResNetSE-100.pt')
model.eval()

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     

In [16]:
for idx, inputs in enumerate(test_loader):
    inputs = inputs.to(device)
    #print(inputs.shape)
    outputs = model(inputs)
    pred = outputs.argmax(dim=1, keepdim=True)
    print(pred)
    pred_int = pred.item()
    result.iloc[idx,1] = pred_int
result

tensor([[13]], device='cuda:0')
tensor([[14]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[11]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[19]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[15]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[6]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[7]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[14]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[18]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[18]], device='cuda:0')
tensor([[14]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[17]], devic

,FileName,Predicted_ID
0,TEST001.ogg,13
1,TEST002.ogg,14
2,TEST003.ogg,12
3,TEST004.ogg,3
4,TEST005.ogg,11
...,...,...
184,TEST185.ogg,4
185,TEST186.ogg,3
186,TEST187.ogg,11
187,TEST188.ogg,11


In [17]:
for i in range(189):
    if(result.iloc[i,1]==0):
        result.iloc[i,1]=20
result

,FileName,Predicted_ID
0,TEST001.ogg,13
1,TEST002.ogg,14
2,TEST003.ogg,12
3,TEST004.ogg,3
4,TEST005.ogg,11
...,...,...
184,TEST185.ogg,4
185,TEST186.ogg,3
186,TEST187.ogg,11
187,TEST188.ogg,11


In [18]:
result.to_csv('test_easy7.csv', index=False)